ASAP CRN Metadata compilation

# Team Hardy. ASAP CRN Metadata scrubbing

confirm Team Hardy scRNAseq metadata
9 Oct 2024
Andy Henrie




In [1]:
import pandas as pd
from pathlib import Path
import os, sys

sys.path.append(os.path.abspath((os.path.join(os.getcwd(), 'src/crn_utils'))))

from util import read_CDE, NULL, prep_table, read_meta_table
from validate import validate_table, ReportCollector
from update_schema import v1_to_v2, v2_to_v3_PMDBS, create_upload_medadata_package

%load_ext autoreload
%autoreload 2


Streamlit NOT successfully imported


## CDEs
load the relavent CDEs

In [2]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


> SANITY CHECK: verify reading from google doc works.

```python
CDEv1_ = read_CDE("v1")
CDEv2_ = read_CDE("v2.1")
CDEv3_ = read_CDE("v3.0")
```


## Clean V1 Table
write clean metadata tables according to CDE v1

### Team Hardy

In [3]:
## convert 
team = "hardy"
collection = "sn_pmdbs"

root_path = Path.home() / ("Projects/ASAP/data/teams")

metadata_path = root_path / f"{team}/{collection}/metadata/"
og_path = metadata_path / "og"


# v1_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'CLINPATH', 'SAMPLE']

# dfs = {}
# for table in v1_tables:
#     dfs[table] =read_meta_table(og_path / f"{table}.csv")


STUDY = read_meta_table(og_path / "STUDY.csv")
PROTOCOL = read_meta_table(og_path / "PROTOCOL.csv")
SUBJECT = read_meta_table(og_path / "SUBJECT.csv")
CLINPATH = read_meta_table(og_path / "CLINPATH.csv")
SAMPLE = read_meta_table(og_path / "SAMPLE.csv")


metadata_version = "v1"

METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"

In [4]:
STUDY

,name,value
0,project_name,Understanding mechanisms of Parkinson's diseas...
1,project_dataset,Hardy snRNA-seq
2,project_description,Genetic analysis has identified many risk gene...
3,ASAP_team_name,TEAM-HARDY
4,ASAP_lab_name,Ryten Lab
5,PI_full_name,Mina Ryten
6,PI_email,mina.ryten@ucl.ac.uk
7,contributor_names,"Aine Fairbrother-Browne, Jonathan Brenton, Mel..."
8,submitter_name,Aine Fairbrother-Browne
9,submitter_email,aine.fairbrother-browne.18@ucl.ac.uk


In [5]:

# fix STUDY formatting
tmp = pd.DataFrame()
tmp = STUDY[["name","value"]].transpose().reset_index().drop(columns=["index"])
tmp

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NA,NA,NA,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,"Version 1, 09/2023"


In [6]:

tmp.columns = tmp.iloc[0]
STUDY = tmp.drop([0])
STUDY.head()


,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
1,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NA,NA,NA,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,"Version 1, 09/2023"


In [15]:
STUDY['metadata_version_date'] = METADATA_VERSION_DATE
STUDY.reset_index(drop=True, inplace=True)
STUDY.head()

,project_name,project_dataset,project_description,ASAP_team_name,ASAP_lab_name,PI_full_name,PI_email,contributor_names,submitter_name,submitter_email,...,other_funding_source,publication_DOI,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,metadata_version_date
0,Understanding mechanisms of Parkinson's diseas...,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,TEAM-HARDY,Ryten Lab,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,aine.fairbrother-browne.18@ucl.ac.uk,...,NA,NA,NA,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,v1_20241018


In [16]:
PROTOCOL

,sample_collection_summary,cell_extraction_summary,lib_prep_summary,data_processing_summary,github_url,protocols_io_DOI,other_reference
0,"This dataset contains cortical regions only, p...",From protocols.io: This protocol is used to is...,'Nuclei were extracted from homogenised post-m...,Cell ranger was used to convert raw sequencing...,Raw to fastq to mapped: https://github.com/RHR...,Nuclear extraction protocol: 10.17504/protocol...,NA


In [17]:
# # fix PROTOCOL formatting
# tmp = pd.DataFrame()
# tmp = PROTOCOL[["name","value"]].transpose().reset_index().drop(columns=["index"])
# tmp.columns = tmp.iloc[0]
# PROTOCOL = tmp.drop([0])
PROTOCOL.reset_index(drop=True, inplace=True)
PROTOCOL.head()


,sample_collection_summary,cell_extraction_summary,lib_prep_summary,data_processing_summary,github_url,protocols_io_DOI,other_reference
0,"This dataset contains cortical regions only, p...",From protocols.io: This protocol is used to is...,'Nuclei were extracted from homogenised post-m...,Cell ranger was used to convert raw sequencing...,Raw to fastq to mapped: https://github.com/RHR...,Nuclear extraction protocol: 10.17504/protocol...,NA


In [18]:
SAMPLE[['assay','sequencing_end', 'sequencing_length', 'sequencing_instrument']]

,assay,sequencing_end,sequencing_length,sequencing_instrument
0,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
1,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
2,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
4,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
...,...,...,...,...
3611,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3612,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3613,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000
3614,v3.1 - Dual Index,paired-end,190,Illumina NovaSeq 6000


In [19]:

# force the right sex_ontology_term_id
SAMPLE["organism_ontology_term_id"] = "NCBITaxon:9606"

SAMPLE


# add 'replicate' coding (nans)

,sample_id,source_subject_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id
0,babom_ACG,P2/14,babom,NA,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
1,babom_ACG,P2/14,babom,NA,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
2,babom_ACG,P2/14,babom,NA,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
3,babom_ACG,P2/14,babom,NA,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
4,babom_ACG,P2/14,babom,NA,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3611,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0006088,NA,EFO:0008913,nucleus,NA,NA,NA
3612,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0006088,NA,EFO:0008913,nucleus,NA,NA,NA
3613,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0006088,NA,EFO:0008913,nucleus,NA,NA,NA
3614,zupam_IPL,P78/11,zupam,rep2,2,1,B2,brain,IPL,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0006088,NA,EFO:0008913,nucleus,NA,NA,NA


In [20]:
SAMPLE['replicate'] = SAMPLE['replicate'].replace({NULL: "Rep1", "rep1": "Rep1", "rep2": "Rep2"})

In [21]:
SAMPLE.head()

,sample_id,source_subject_id,subject_id,replicate,replicate_count,repeated_sample,batch,tissue,brain_region,source_RIN,...,sex_ontology_term_id,self_reported_ethnicity_ontology_term_id,disease_ontology_term_id,tissue_ontology_term_id,cell_type_ontology_term_id,assay_ontology_term_id,suspension_type,DV2000,pm_PH,donor_id
0,babom_ACG,P2/14,babom,Rep1,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
1,babom_ACG,P2/14,babom,Rep1,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
2,babom_ACG,P2/14,babom,Rep1,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
3,babom_ACG,P2/14,babom,Rep1,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA
4,babom_ACG,P2/14,babom,Rep1,1,0,B1,brain,ACG,NA,...,PATO:0000383 (female),NA,MONDO:0005180,UBERON:0009835,NA,EFO:0008913,nucleus,NA,NA,NA


In [22]:
CLINPATH['path_braak_asyn']

0       6
1       6
2       6
3       6
4       0
       ..
133    NA
134    NA
135    NA
136    NA
137    NA
Name: path_braak_asyn, Length: 138, dtype: object

In [23]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_braak_nft'] = CLINPATH['path_braak_nft'].replace({"0":"0", 
                                                                "1":"I", 
                                                                "2": "II", 
                                                                "3":"III", 
                                                                "4":"IV", 
                                                                "5":"V", 
                                                                "6":"VI"})


In [24]:

# code family_history as "Not Reported" (currently empty)
CLINPATH['family_history'] = "Not Reported"



# check APOE_e4_status ? currently empty

# `path_autopsy_dx_main`  actually seems good parser might be wrong

# code "at least 4" as "4/5" 

CLINPATH['path_thal'] = CLINPATH['path_thal'].replace({'At least 4':"4/5"})


CLINPATH['path_mckeith'] = CLINPATH['path_mckeith'].replace({'Diffuse neocortical': "Diffuse, neocortical (brainstem, limbic and neocortical involvement)", 
                                                        'Limbic transitional': "Limbic (transitional)" ,
                                                        'Diffuse Neocortical':"Diffuse, neocortical (brainstem, limbic and neocortical involvement)"})


In [25]:

CLINPATH['path_nia_aa_a'] = CLINPATH['path_nia_aa_a'].replace({"0":"A0", 
                                                                    "1":"A1", 
                                                                    "2": "A2", 
                                                                    "3":"A3"})


In [26]:

# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_b'] = CLINPATH['path_nia_aa_b'].replace({"0":"B0", 
                                                                "1":"B1", 
                                                                "2": "B2", 
                                                                "3":"B3"})


# replace 'path_braak_nft' with with string of the numeric. converte nan to ""
CLINPATH['path_nia_aa_c'] = CLINPATH['path_nia_aa_c'].replace({"0":"C0", 
                                                                "1":"C1", 
                                                                "2": "C2", 
                                                                "3":"C3"})



In [27]:



CLINPATH['path_ad_level'] = CLINPATH['path_ad_level'].replace({"No evidence": "No evidence of Alzheimer\'s disease neuropathological change"})





In [28]:
path_autopsy_map = { "Parkinson's disease with dementia": "Parkinson's disease with dementia", 
       "Parkinson's disease": "Parkinson's disease",
       'Control brain':"Control, no misfolded protein or significant vascular pathology", 
       'Pathological ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Control brain / Path ageing': 'Control, no misfolded protein or significant vascular pathology',
       'Argyrophilic grain disease': "Control, Argyrophilic grain disease",
       'Control brain, Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       'Cerebrovascular disease (small vessel)':"Control, Cerebrovascular disease (atherosclerosis)",
       "Control brain, Alzheimer`s disease (intermediate level AD pathological change)":"Alzheimer's disease (intermediate level neuropathological change)",
       'Control brain / Path ageing, CAA':"Control, Cerebrovascular disease (cerebral amyloid angiopathy)"}


In [29]:
CLINPATH['path_autopsy_dx_main'] = CLINPATH['path_autopsy_dx_main'].replace(path_autopsy_map)
# - _*age_at_diagnosis*_:  invalid values 💩'69', '61', 'Not applicable', '71', '60', '57', '79', '87', '67', '75', '50', '35', '44', '42', '58', '80', '72', '65', '48', '74', '62', '36', '77', '43', '56', '55', '37', '66', '64', '49', 'NA'

CLINPATH['age_at_diagnosis'] = CLINPATH['age_at_diagnosis'].replace({"Not applicable": NULL})

In [30]:
# 🚨⚠️❗ **Extra field in CLINPATH: source_subject_id**
# rename source_subject_id to source_sample_id
CLINPATH.rename(columns={"source_subject_id":"source_sample_id"}, inplace=True)


In [31]:
SUBJECT['primary_diagnosis'] = SUBJECT['primary_diagnosis'].replace({"Alzheimer?s disease":"Alzheimer's disease"})

In [32]:
# - _*molecular_source*_:  invalid values 💩'nuclear RNA'
#     - valid ➡️ 'Total RNA', 'PolyA RNA', 'Cytoplasmic RNA', 'Nuclear RNA', 'Genomic DNA', 'Protein', 'Other'
# - _*input_cell_count*_:  invalid values 💩'16483.800000000003', '20418', '16714.643750000003', '17644.912977099237', '17353.600000000002', '17388.800000000003', '17952', '17776', '19712', '16948.800000000003', '17098', '18656', '17265.600000000002', '12223.199999999999', '14520', '18128', '17933.2', '12289.199999999999', '12051.599999999999', '13860', '13970', '18304', '17564.879999999997', '18480', '17916.800000000003', '19855.07096774194', '17368.707692307697', '16766', '17529.600000000002', '19360', '23100', '15180', '18100.800000000003', '19536', '18832', '13094.4', '18601.559999999998', '13194.72', '13226.4', '11893.199999999999', '16632', '20400', '21800', '17555.9', '15312', '12249.599999999999', '18426', '23738.000000000004', '17920', '18347.28', '18503.76', '16859', '19680', '18163.2', '20856', '15048', '14124', '17600', '12381.599999999999', '16566.800000000003', '19422', '21120', '17300.800000000003', '21472.100000000002', '45500.600000000006', '17001.600000000002', '17314', '10942.8', '9292.8', '20064', '18008.199999999997', '17476.800000000003', '12157.199999999999', '17688', '13570.656', '13572.24', '18249.48', '16684.800000000003', '13678.5', '17716.83', '18458', '25600', '23800', '13195.775999999998', '13190.099999999999', '18240', '17762', '19920', '18445.079999999998', '17738.6', '16772.800000000003', '17477.7', '19520', '19754', '17211.828571428574', '14388', '15582', '22400', '22000', '18260', '14552.999999999998', '13332', '19928.300000000003', '19256', '19008', '18560', '17760', '20617.2', '18053.879999999997', '27365.100000000002', '17885.2'
#     - valid ➡️ int or NULL ('NA')
# - _*sequencing_end*_:  invalid values 💩'paired-end'
#     - valid ➡️ 'Single', 'Paired-end'
# - _*sequencing_length*_:  invalid values 💩'190'
#     - valid ➡️ '25', '50', '100', '150'
# - _*technology*_:  invalid values 💩'sN'
#     - valid ➡️ 'SN', 'SC', 'Spatial', 'Bulk', 'Seq', 'Array'
# - _*adjustment*_:  invalid values 💩'raw'
#     - valid ➡️ 'Raw', 'Processed'
# - _*content*_:  invalid values 💩'reads'
#     - valid ➡️ 'Counts', 'Probabilities', 'Genotypes', 'Dosages', 'Reads'

SAMPLE = prep_table(SAMPLE, CDEv1)

In [33]:
v1_path = metadata_path / "v1"
STUDY.to_csv(v1_path / f"STUDY.csv", index=False)
SAMPLE.to_csv(v1_path / f"SAMPLE.csv", index=False)
SUBJECT.to_csv(v1_path / f"SUBJECT.csv", index=False)
PROTOCOL.to_csv(v1_path / f"PROTOCOL.csv", index=False)
CLINPATH.to_csv(v1_path / f"CLINPATH.csv", index=False)

In [34]:
CDE = CDEv1
tables = CDE['Table'].unique()
v1_path = metadata_path / "v1"

dfs = {}
for table in tables:
    df = read_meta_table(v1_path / f"{table}.csv")
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()
    dfs[table] = full_table
    # df.to_csv(v1_path / f"{table}.csv", index=False)

recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **4 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- DUA_version: 1/1 empty rows (REQUIRED)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **3 Fields with empty (NULL) values:**

	- race: 64/64 empty rows (REQUIRED)

	- ethnicity: 64/64 empty rows (REQUIRED)

	- duration_pmi: 1/64 empty rows (REQUIRED)
No invalid entries found in Enum fields.

recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int
🚨⚠️❗ **Missing Required Fields in SAMPLE: source_sample_id**
🚨⚠️❗ **Missing Optional Fields in SAMPLE: DV200**
🚨⚠️❗ **9 Fields with empty (NULL) values:**

	- source_RIN: 3616/3616 empty rows (REQUIRED)

	- header: 3616/3616 empty rows (REQUIRED)

	- annotation: 3616/3616 

In [35]:
# make tables conform to CDE and save extra columns as "auxiliary"

for table in tables:
    df = dfs[table]
    schema = CDE[CDE['Table'] == table]
    valid_fields = schema['Field'].unique()
    df_out = df[valid_fields]
    aux_fields = set(df.columns) - set(valid_fields)
    if aux_fields:
        df_aux = df[list(aux_fields)]
        df_aux.to_csv(v1_path / f"{table}_auxiliary.csv", index=False)
        print(f"Saved {table}_auxiliary.csv")
    df_out.to_csv(v1_path / f"{table}.csv", index=False)
    

Saved SAMPLE_auxiliary.csv


## Update the table to v2

In [36]:
v2_path = metadata_path / "v2"
# rationalize the dataset id
team_dataset_id = "sc_pmdbs"

v2_tables, aux_tables = v1_to_v2(v1_path, v2_path, CDEv1, CDEv2, team_dataset_id=team_dataset_id)



recoding number_of_brain_samples as int
recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int


In [37]:
v2_tables['STUDY']


,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,publication_PMID,number_of_brain_samples,brain_regions,types_of_samples,PI_ORCHID,PI_google_scholar_id,DUA_version,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-HARDY,Ryten Lab,Understanding mechanisms of Parkinson's diseas...,sc_pmdbs,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,...,NA,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,0000-0001-9520-6957,https://scholar.google.co.uk/citations?user=lt...,NA,Https://github.com/RHReynolds/asap-pd-progress...,v2.1_20241018,NA


validate v2 tables


In [38]:
CDE = CDEv2
for table,df in v2_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()


recoding number_of_brain_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **5 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- DUA_version: 1/1 empty rows (REQUIRED)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *SUBJECT* table.
🚨⚠️❗ **15 Fields with empty (NULL) values:**

	- AMPPD_id: 74/74 empty rows (REQUIRED)

	- GP2_id: 2/74 empty rows (REQUIRED)

	- race: 74/74 empty rows (REQUIRED)

	- ethnicity: 74/74 empty rows (REQUIRED)

	- last_diagnosis: 74/74 empty rows (R

In [39]:
v3_meta_tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']

f"{v3_meta_tables}"

"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DATA', 'CLINPATH', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']"

In [45]:
v3_path = metadata_path / "v3"

v3_tables, aux_tables = v2_to_v3_PMDBS(v2_path, v3_path, CDEv2, CDEv3)

recoding number_of_brain_samples as int
recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
recoding replicate_count as int
recoding repeated_sample as int
recoding input_cell_count as int
recoding replicate_count as int
recoding repeated_sample as int


In [46]:
CDE = CDEv3
for table,df in v3_tables.items():
    schema = CDE[CDE['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table, report = validate_table(df.copy(), table, schema, report)
    report.print_log()

recoding number_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **6 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- DUA_version: 1/1 empty rows (REQUIRED)

	- PI_ORCID: 1/1 empty rows (OPTIONAL)

	- alternate_dataset_id: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- race: 64/64 empty rows (REQUIRED)
No invalid entries found in Enum fields.

recoding replicate_count as int
recoding repeated_sample as int
All required fields are present in *SAMPLE* table.
🚨⚠️❗ **12 Fields with empty (NULL) values:**

	- source_sample_id: 138/138

In [47]:
STUDY = v3_tables['STUDY']
STUDY

,ASAP_team_name,ASAP_lab_name,project_name,team_dataset_id,project_dataset,project_description,PI_full_name,PI_email,contributor_names,submitter_name,...,number_samples,sample_types,types_of_samples,DUA_version,metadata_tables,PI_ORCID,PI_google_scholar_id,preprocessing_references,metadata_version_date,alternate_dataset_id
0,TEAM-HARDY,Ryten Lab,Understanding mechanisms of Parkinson's diseas...,sc_pmdbs,Hardy snRNA-seq,Genetic analysis has identified many risk gene...,Mina Ryten,mina.ryten@ucl.ac.uk,"Aine Fairbrother-Browne, Jonathan Brenton, Mel...",Aine Fairbrother-Browne,...,128,"Inferior Parietal Lobule (IPL), Anterior Cingu...",Late stage (Braak 5-6) PD and control post-mor...,NA,"['STUDY', 'PROTOCOL', 'SUBJECT', 'SAMPLE', 'DA...",NA,https://scholar.google.co.uk/citations?user=lt...,Https://github.com/RHReynolds/asap-pd-progress...,v3.0_20241019,NA


### Create metadata package


In [48]:
export_path = root_path / f"{team}"

create_upload_medadata_package(export_path, v3_tables)

________

In [49]:
v3_tables['CONDITION']

Field,condition_id,intervention_name,intervention_id,protocol_id,intervention_aux_table
0,idiopathic_pd,Case-Control,Case,NA,NA
1,no_pd_nor_other_neurological_disorder,Case-Control,Control,NA,NA
2,alzheimer's_disease,Case-Control,Other,NA,NA
3,other_neurological_disorder,Case-Control,Other,NA,NA


Transfer cleaned metadata to raw buckets 




## transfer metadata Hardy 

In [68]:
# Hardy
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/hardy-credentials.json 


Activated service account credentials for: [raw-admin-hardy@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [69]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-hardy/"


        20  2023-08-15T15:13:44Z  gs://asap-raw-data-team-hardy/test-raw-hardy.txt#1692112424343862  metageneration=1
        19  2023-08-17T09:58:09Z  gs://asap-raw-data-team-hardy/testfile.txt#1692266289818249  metageneration=1
                                 gs://asap-raw-data-team-hardy/fastqs/
                                 gs://asap-raw-data-team-hardy/metadata/
                                 gs://asap-raw-data-team-hardy/ogmetadata/
TOTAL: 2 objects, 39 bytes (39 B)


## fix bucket file structure to match the prescribed convention

change "fastq" -> "fastqs"

In [4]:

!gsutil -u dnastack-asap-parkinsons mv "gs://asap-raw-data-team-hardy/fastq" "gs://asap-raw-data-team-hardy/fastqs"


Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I1_001.fastq.gz...
Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I2_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_I2_001.fastq.gz...
Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Removing gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_R1_001.fastq.gz...
Copying gs://asap-raw-data-team-hardy/fastq/220520_A01366_0202_AHL5JVDSX3/HL5JVDSX3/fikon_IPL_S12_L001_R2_001.fastq.gz [Content-Type=application/

In [6]:
!gsutil -u dnastack-asap-parkinsons ls "gs://asap-raw-data-team-hardy/fastqs" 

gs://asap-raw-data-team-hardy/fastqs/all_fastq.txt
gs://asap-raw-data-team-hardy/fastqs/220520_A01366_0202_AHL5JVDSX3/
gs://asap-raw-data-team-hardy/fastqs/220706_A01897_0009_AHMYFTDSX3/
gs://asap-raw-data-team-hardy/fastqs/220706_A01897_0010_BHMYM5DSX3/
gs://asap-raw-data-team-hardy/fastqs/220725_A01897_0013_BHLNTGDSX3/
gs://asap-raw-data-team-hardy/fastqs/220725_A01897_0014_AHN3MVDSX3/
gs://asap-raw-data-team-hardy/fastqs/220811_A01897_0015_BH3NFNDSX5/
gs://asap-raw-data-team-hardy/fastqs/220811_A01897_0016_AHF7KJDMXY/
gs://asap-raw-data-team-hardy/fastqs/221013_A01897_0034_AHGFTNDMXY/


In [70]:

# !gsutil -u dnastack-asap-parkinsons cp -r ./clean/team-Hardy/v2_20231128  "gs://asap-raw-data-team-hardy/metadata/v2"
!gsutil -u dnastack-asap-parkinsons cp -r "./clean/team-Hardy/v2_20231201/*.csv"  "gs://asap-raw-data-team-hardy/metadata/v2"


Copying file://./clean/team-Hardy/v2_20231201/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/DATA.csv [Content-Type=text/csv]...
- [4 files][552.2 KiB/552.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Hardy/v2_20231201/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/PROTOCOL.csv [Content-Type=text/csv]...
\ [6 files][556.5 KiB/556.5 KiB]                                                
Operation completed over 6 objects/556.5 KiB.                                   

### copy to workflow-dev bucket

First copy each set of metadata locally ...

In [72]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [73]:
!gsutil  cp -r "./clean/team-Hardy/v2_20231201/*.csv" "gs://asap-workflow-dev/metadata/v2/hardy"

Copying file://./clean/team-Hardy/v2_20231201/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SUBJECT.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/SAMPLE.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/DATA.csv [Content-Type=text/csv]...
- [4 files][552.2 KiB/552.2 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Hardy/v2_20231201/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Hardy/v2_20231201/PROTOCOL.csv [Content-Type=text/csv]...
\ [6 files][556.5 KiB/556.5 KiB]                                                
Operation completed over 6 objects/556.5 KiB.                                   

## check file md5s

In [35]:
from utils.checksums import extract_md5_from_details, extract_md5_from_details2


In [35]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/hardy-credentials.json  



Activated service account credentials for: [raw-admin-hardy@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [36]:

# !gcloud storage hash "gs://asap-raw-data-team-hardy/**/*.gz"  --skip-crc32c --hex  --billing-project dnastack-asap-parkinsons > hardy_hexhash.log

!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-hardy/**/*.gz" > hardy_hexhash.log


using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [37]:
bucket_files_md5 = extract_md5_from_details2("hardy_hexhash.log")



checksum = DATAv2[['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5'].str.strip()
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)




/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_56961/3763221414.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] = checksum['file_MD5'].str.strip()
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_56961/3763221414.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)


In [1]:
! pip freeze

anyio @ file:///home/conda/feedstock_root/build_artifacts/anyio_1693488585952/work
appnope @ file:///home/conda/feedstock_root/build_artifacts/appnope_1649077682618/work
argcomplete==1.12.3
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///Users/runner/miniforge3/conda-bld/argon2-cffi-bindings_1695386808436/work
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1696128962909/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
async-lru @ file:///home/conda/feedstock_root/build_artifacts/async-lru_1690563019058/work
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1683424013410/work
Babel @ file:///home/conda/feedstock_root/build_artifacts/babel_1698174530262/work
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backport

In [38]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()


[]

In [46]:
checksum.head()

,file_name,file_MD5,check
0,babom_ACG_S20_L008_R1_001.fastq.gz,F37738d04879c1cb8c380920964dad4d,f37738d04879c1cb8c380920964dad4d
1,babom_ACG_S20_L008_I1_001.fastq.gz,F3053c59a33ce48cbea6a602dcc491c7,f3053c59a33ce48cbea6a602dcc491c7
2,babom_ACG_S20_L008_R2_001.fastq.gz,69fe97e968dc1cb782a3d995918b5bc8,69fe97e968dc1cb782a3d995918b5bc8
3,babom_ACG_S20_L008_I2_001.fastq.gz,4eeee300289a4fd18169dddbbfd8ad72,4eeee300289a4fd18169dddbbfd8ad72
4,babom_ACG_S20_L007_R1_001.fastq.gz,7b321fc0e55ffbd496a241dcb0746def,7b321fc0e55ffbd496a241dcb0746def


In [ ]:
# these are actually matches... probably unstripped line endings.